In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torch
import torch.nn.functional as F

In [2]:
# load the quality overview
quality = pd.read_csv("~/MasterThesis/quality.csv")

# More info about the sequences
sequences = pd.read_csv("~/MasterThesis/sequences.csv")

# filter based on the table above
useable = quality[
    #(quality["Vessel in Frame"] <3) &
    #(quality['LM configuration'] < 4) &
    #(quality['Vein contrast/cropping error'] < 3) &
    #(quality['Artery contrast'] < 3) &
    #(quality['Vein boundary'] < 3) &
    #(quality['Artery boundary'] < 3) &
    #(quality['Gain'] == 1) &
    #(quality['Artefacts'] < 3) &
    #(quality['Movement'] < 3) &
    (quality['Total'] < 20)
]
useable.rename(columns={'ID': 'Id'}, inplace=True)
useable

/home/nicke/miniconda3/envs/flowenv/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Id,Vessel in Frame,LM configuration,Vein contrast/cropping error,Artery contrast,Vein boundary,Artery boundary,Gain,Depth,Artefacts,Movement,Total
0,113.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,17
1,116.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,11
2,119.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,12
3,120.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,16
4,121.0,1.0,2.0,3.0,2.0,3.0,1.0,1.0,1.0,2.0,1.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...
4762,11276.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,1.0,17
4763,11277.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,14
4764,11278.0,1.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,16
4765,11280.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,12


In [4]:
train_ids = [124,
413,
419,
524,
545,
209,
216,
456,
420,
546,
214,
304,
316,
418,
523,
543,
544,
549,
601,
610,
626,
667,
668,
692,
752,
772,
773,
1046,
1048,
1092,
1122,
1123,
1145,
172,
176,
177,
 184,
 211,
 219,
 246,
 307,
 341,
 349,
 353,
 354,
 384,
 402,
 408,
 416,
 426,
 442,
 452,
 457,
 467,
 486,
 515,
 521,
 526,
 542,
 553,
 563,
 573,
 578,
 584,
 599,
 1055,
 1065,
 1072,
 1082,
 1091,
 1097,
 1098,
 1101,
 1111,
 1128,
 1131,
 1141,
 1146,
 1148,
 1149,
 1150,
 1156,
 1164,
 1166,
 1168,
 1169,
 1301,
 1311,
 1314,
 4,
 9,
 10,
 12,
 13,
 17,
 20,
 21,
 25,
 28,
 29,
 33,
 34,
 36,
 42,
 58,
 59,
 61,
 67,
 68,
 70,
 75,
 78,
 79,
 84,
 100,
 106,
 107,
 109,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 130,
 135,
 136,
 137,
 140,
 146,
 147,
 151,
 169,
 2709,
 2709,
 2710,
 2710,
 2713,
 2713,
 2714,
 2714,
 2733,
 2733,
 2754,
 2758,
 2761,
 2765,
 2765,
 2766,
 4774,
 4775,
 4778,
 4781,
 4796,
 4798,
 4799,
 4801,
 4804,
 4808,
 4810,
 4813,
 4814,
 4825,
 4834,
 4856,
 2185,
 2187,
 2199,
 1627,
 1643,
 1354,
 1352,
 1366,
 844,
 1367,
 1667,
 913,
 1668,
 917,
 918,
 1669,
 1671,
 1673,
 1676,
 931,
 934,
 1015,
 1723,
 1452,
 1464,
 1462,
 1526,
 1616,
 2065,
 2141,
 2143,
 977,
 1693,
 1715,
 1733,1735,1741,1497,1506,1519,1620,986,929,968,859,861,906,908,910,956,962,919,903,905,921,922,943,944,946,947,965,975,997,998,1778,2829,2830,2831,2732,2736,2841,2866,2867,2868,2874,2875,2764, 2885]

In [8]:
test_IDs = useable[~useable.Id.isin(train_ids)].Id.to_numpy()
test_IDs

array([  119.,   129.,   132., ..., 11278., 11280., 11286.])

In [9]:
path_to_data = "/share/data_ultraschall/compressions"

In [18]:
ids_with_segs = []
for test_id in test_IDs:
    id_path = os.path.join(path_to_data, str(int(test_id)), "segmentations", "1")
    if os.path.isdir(id_path):
        if len(os.listdir(id_path)) != 0:
            ids_with_segs.append(test_id)

ids_with_segs

[1680.0,
 1702.0,
 1703.0,
 2190.0,
 4821.0,
 4833.0,
 4839.0,
 4841.0,
 4853.0,
 3714.0,
 3753.0,
 3822.0,
 3933.0,
 3935.0,
 3944.0,
 4002.0,
 4113.0,
 4155.0,
 4160.0,
 4181.0,
 4183.0,
 4186.0,
 4240.0,
 4242.0,
 4300.0,
 4333.0,
 4397.0,
 4608.0,
 4616.0,
 4740.0,
 4770.0,
 4824.0,
 4936.0,
 4954.0,
 5028.0,
 5065.0,
 5070.0,
 5087.0,
 5114.0,
 5129.0,
 5167.0,
 5175.0,
 5178.0,
 5209.0,
 5218.0,
 5220.0,
 5311.0,
 4633.0,
 3296.0,
 3297.0,
 3299.0,
 3302.0,
 3304.0,
 3305.0,
 3308.0,
 3548.0,
 3314.0,
 3496.0,
 3318.0,
 3321.0,
 3322.0,
 3550.0,
 3552.0,
 3326.0,
 3553.0,
 3500.0,
 3328.0,
 3329.0,
 3333.0,
 3335.0,
 3336.0,
 3555.0,
 3339.0,
 3343.0,
 3345.0,
 3350.0,
 3351.0,
 3352.0,
 3353.0,
 3354.0,
 3356.0,
 3357.0,
 3358.0,
 3556.0,
 3360.0,
 3364.0,
 3368.0,
 3369.0,
 3370.0,
 3376.0,
 3378.0,
 3381.0,
 3382.0,
 3387.0,
 3394.0,
 3397.0,
 3401.0,
 3514.0,
 3403.0,
 3405.0,
 3407.0,
 3518.0,
 3410.0,
 3411.0,
 3412.0,
 3419.0,
 3424.0,
 3425.0,
 3426.0,
 3430.0,
 3431.0,
 

In [54]:
selected_ids = np.random.choice(ids_with_segs,30, replace=False)
selected_ids = selected_ids.astype(np.int32)
selected_ids

array([3485, 2232, 4333, 3753, 3455, 5687, 3360, 3672, 3357, 2098, 4242,
       3933, 3628, 3538, 5209, 4160, 3352, 3586, 3433, 3644, 3387, 3663,
       3424, 2219, 3649, 3335, 4186, 3397, 3683, 1703], dtype=int32)

In [57]:
len(np.unique(selected_ids))

30

In [2]:
# load the ids of available patients
available = pd.read_csv("/home/nicke/MasterThesis/available_US_probands.csv")
available = available.drop('Unnamed: 0', axis=1)
available = available[available['Anatomy'] != 'BACKGROUND']
available

,Id,Patient,Device,Leg,Anatomy,Landmark,Datatype,Vessel in Frame,LM configuration,Vein contrast/cropping error,Artery contrast,Vein boundary,Artery boundary,Gain,Depth,Artefacts,Movement,Total
0,4,1,guys,LT,SFV1,LM5,healthy,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,15
1,9,2,guys,RT,SFV1,LM5,healthy,1.0,1.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,17
2,10,2,guys,RT,SFV2,LM6,healthy,1.0,1.0,3.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,19
3,12,2,guys,LT,SFV1,LM5,healthy,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,13
4,13,2,guys,LT,SFV2,LM6,healthy,1.0,2.0,2.0,2.0,3.0,3.0,1.0,1.0,2.0,1.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,5394,228,philips,LT,CFV + SFA / PFA,LM3,healthy,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,14
469,5396,228,philips,LT,GSV / CFV + SFA / PFA,"LM1,LM3",healthy,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,17
470,5682,280,philips,RT,SFV + SFA,LM6,healthy,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,16
471,5687,282,philips,RT,SFV + SFA,LM6,healthy,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,12


In [3]:
path_to_data = '/share/data_ultraschall/compressions'

In [5]:
landmarks = pd.read_csv('/home/nicke/MasterThesis/landmarks.csv')
landmarks = landmarks[(landmarks['Start Frames'] != '[]') & (landmarks['End Frames'] != '[]')& (landmarks['End Frames'] != 'DNC')]
landmarks

,Id,Landmark,Total Frames,Start Frames,End Frames
0,152,"LM1,LM2",229,[7],[193]
1,156,"LM1,LM2,LM3",258,[22],[187]
2,160,"LM1,LM2",223,[14],[144]
3,24,"LM1, LM2, LM3",90,[0],[80]
4,52,LM1,65,[11],[47]
...,...,...,...,...,...
4987,2764,"LM9, LM10, LM8, LM8, LM10, LM9",187,"[1, 31, 72, 99, 133, 152]","[33, 73, 101, 135, 153, 187]"
4988,2884,LM3,39,[5],[23]
4989,2765,LM8,83,[27],[54]
4990,2885,"LM4,LM5",39,[7],[27]


In [7]:
data = landmarks.merge(available, on='Id')
id_list = (data.Id.values).astype(str)

In [8]:
def load_image_and_seg(path, image):
    
    # load image with PIL and normalize
    img = np.array(Image.open(os.path.join(path,"frames", image))) / 255
    #print(os.path.join(path,"frames", image))
    #print(os.path.join(path,"segmentations", "1", image))
    
    # load seg and normalize over the labels
    seg =np.array(Image.open(os.path.join(path,"segmentations", "1", image))) / 200
    
    return img, seg

def get_image_seg_pairs(prob_id):
    
    
    print(f"Working on id: {prob_id}")
    all_frames = sorted(os.listdir(os.path.join(path_to_data,prob_id,"frames")))

    first_available_frame = np.fromstring(landmarks[landmarks['Id']== int(prob_id)]['Start Frames'].iat[0].strip(']['), sep=',', dtype=int)
    last_available_frame = np.fromstring(landmarks[landmarks['Id']== int(prob_id)]['End Frames'].iat[0].strip(']['), sep=',', dtype=int)
    file_pairs = []
    for j,f_frame in enumerate(first_available_frame):
        i = 6
        while f_frame + i < last_available_frame[j]:
            file_pairs.append([all_frames[f_frame], all_frames[f_frame+i]])
            i = i+6
            if len(file_pairs) > 6:
                break
    
    frame_pairs = []
    seg_pairs = []
    
    # load the seg and frame for fixed and moving
    for fixed_file, moving_file in file_pairs:
        
        fixed, fixed_seg = load_image_and_seg(os.path.join(path_to_data, prob_id), fixed_file)
        moving, moving_seg = load_image_and_seg(os.path.join(path_to_data, prob_id), moving_file)
        
        if fixed_seg.max() == 0:
            continue
        if moving_seg.max() == 0:
            continue
        
        
        # and store them together
        frame_pairs.append([fixed,moving])
        seg_pairs.append([fixed_seg,moving_seg])
    
    return np.array(frame_pairs), np.array(seg_pairs)    
    

In [ ]:
frames = []
segs = []
ids = []
for prob_id in id_list:

    frame_pairs, seg_pairs = get_image_seg_pairs(prob_id)
    
    for pair in frame_pairs:
        frames.append(pair)
    for pair in seg_pairs:
        segs.append(pair)